In [ ]:
! git clone https://github.com/qmlcode/tutorial.git

In [ ]:
%config IPCompleter.greedy=True

In [1]:
%cd tutorial
! ls

/home/jan/projects/qmlnet/tutorial
exercise_2_1.py  exercise_2_4.py  modelnew.h5  QML_Tutorial.ipynb
exercise_2_2.py  hof_qm7.txt	  __pycache__  QML_Tutorial_sklearn.ipynb
exercise_2_3.py  LICENSE	  qm7	       tutorial_data.py


In [2]:
#!/usr/bin/env python
from __future__ import print_function
import numpy as np

import qml
from qml.kernels import gaussian_kernel
from qml.math import cho_solve

from tutorial_data import compounds
from tutorial_data import energy_pbe0
from tutorial_data import energy_delta

if __name__ == "__main__":

    # For every compound generate a coulomb matrix
    for mol in compounds:

        mol.generate_coulomb_matrix(size=23, sorting="row-norm")
        # mol.generate_bob(size=23, asize={"O":3, "C":7, "N":3, "H":16, "S":1})
            

    # Make a big 2D array with all the 
    X = np.array([mol.representation for mol in compounds], dtype=np.float32)
    energy_pbe0 = np.array(energy_pbe0,  dtype=np.float32)
    # X = np.array([mol.bob for mol in compounds])

    print(energy_pbe0)

    # Assign 1000 first molecules to the training set
    X_training = X[:1000]
    Y_training = energy_pbe0[:1000]

    # Y_training = energy_delta[:1000]

    # Assign 1000 first molecules to the training set
    X_test = X[-1000:]
    Y_test = energy_pbe0[-1000:]
    # Y_test = energy_delta[-1000:]
   
    # Calculate the Gaussian kernel
    sigma = 100 #700.0
    K = gaussian_kernel(X_training, X_training, sigma)
    print(K)

    # Add a small lambda to the diagonal of the kernel matrix
    K[np.diag_indices_from(K)] += 1e-8

    # Use the built-in Cholesky-decomposition to solve
    alpha = cho_solve(K, Y_training) 

    #print(alpha)

    # Assign 1000 last molecules to the test set
    X_test = X[-1000:]
    Y_test = energy_pbe0[-1000:]

    # calculate a kernel matrix between test and training data, using the same sigma
    Ks = gaussian_kernel(X_test, X_training, sigma)

    # Make the predictions
    Y_predicted = np.dot(Ks, alpha)

    # Calculate mean-absolute-error (MAE):
    print(np.mean(np.abs(Y_predicted - Y_test)))


[-1633.28 -1212.67 -1531.73 ... -1728.7  -1506.76 -1866.88]
[[1.         0.8152882  0.87350271 ... 0.90991951 0.82913994 0.77672171]
 [0.8152882  1.         0.81460855 ... 0.84226491 0.97323939 0.78838225]
 [0.87350271 0.81460855 1.         ... 0.8913725  0.83339596 0.78083487]
 ...
 [0.90991951 0.84226491 0.8913725  ... 1.         0.86685092 0.88326791]
 [0.82913994 0.97323939 0.83339596 ... 0.86685092 1.         0.80932061]
 [0.77672171 0.78838225 0.78083487 ... 0.88326791 0.80932061 1.        ]]
15.00718369095469


In [4]:
len(X_training)

1000

In [ ]:
%load_ext tensorboard

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import datetime
from datetime import datetime
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.layers import (Input, Convolution1D, Dense, MaxPooling1D,MaxPooling2D,
                                    Flatten, Dropout, Activation, average,
                                    BatchNormalization, Reshape, Conv2D)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, QuantileTransformer

from keras.wrappers.scikit_learn import KerasRegressor



"""
X_training  = tf.convert_to_tensor(X_training)
X_test      = tf.convert_to_tensor(X_test)
Y_training  = tf.convert_to_tensor(Y_training)
Y_test      = tf.convert_to_tensor(Y_test)
"""




def create_model(optimizer, dense_nparams, lr, decay, LAMBDA, filters): #, ):
    #filters = 50
    

    init = 'he_normal'
    
    
    regu = tf.keras.regularizers.l2(LAMBDA)


    model = Sequential()
    model.add(Dense(276, input_dim=276, kernel_initializer=init,kernel_regularizer=regu ,activation='linear'))
    #276 = 12 x 23 
    #BatchNormalization()
    
    Reshape((12, 23, 1), input_shape=(276,))
    Conv2D(filters, 5, activation = 'selu', padding = 'same', input_shape= [12, 23, 1] ),
    MaxPooling2D(2)
    Conv2D(filters*2, 5, activation = 'selu', padding = 'same' ),
    Conv2D(filters*2, 5, activation = 'selu', padding = 'same' ),
    MaxPooling2D(2)
    Conv2D(filters*3, 5, activation = 'selu', padding = 'same' ),
    Conv2D(filters*3, 5, activation = 'selu', padding = 'same' ),
    MaxPooling2D(2)

    Flatten()
    #BatchNormalization()
    #model.add(exponential_layer)
    model.add(Dense(dense_nparams, kernel_initializer=init, activation='selu'))
    BatchNormalization()
    #Dropout(rate=0.8)
    model.add(Dense(dense_nparams, kernel_initializer=init, activation='selu'))
    BatchNormalization()
    #Dropout(rate=0.5)
    model.add(Dense(dense_nparams, kernel_initializer=init, activation='selu'))
    BatchNormalization()
    #Dropout(rate=0.4)
    model.add(Dense(dense_nparams, kernel_initializer=init, activation='selu'))
    BatchNormalization()
    model.add(Dense(dense_nparams, kernel_initializer=init, activation='selu'))
    BatchNormalization()
    #Dropout(rate=0.1)
    model.add(Dense(dense_nparams, kernel_initializer=init, activation='selu'))
    BatchNormalization()
    #Dropout(rate=0.1)
    model.add(Dense(dense_nparams, kernel_initializer=init, activation='linear'))
    model.add(Dense(1, activation='linear', name='Output_Energy'))

    opt = Adam(lr=lr ,decay=decay)
    model.compile(loss='mae', optimizer=opt)
    
    #model.summary()
    return model

    
    

#epochs = 300 
kears_estimator = KerasRegressor(build_fn=create_model , epochs=400, batch_size=25, verbose=0)


'''
GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf'
'''

param_grid = {
    'dense_nparams': [300, 400],
    'optimizer':['Adam'], 'lr': [1e-6, 1e5, 1e-4, 1e-3], 'decay': [1e-6, 1e-5,1e-4, 1e-3], 'LAMBDA':  [1e-7, 1e-6, 1e-5], 'filters':  [10, 50, 100] }

#65.587196 using {'decay': 0.0001, 'dense_nparams': 500, 'lr': 0.0001, 'optimizer': 'Adam'}

kfold_splits = 3

#RandomizedSearchCV

search = GridSearchCV(estimator=kears_estimator,  param_grid=param_grid,cv=kfold_splits, return_train_score=True, verbose=10)
#RandomizedSearchCV(model, space, n_iter=500, scoring='accuracy', n_jobs=-1, cv=cv, random_state=1)


my_callbacks =  [keras.callbacks.EarlyStopping(patience=30, monitor="val_loss"), keras.callbacks.TerminateOnNaN() ]
#https://medium.com/@am.benatmane/keras-hyperparameter-tuning-using-sklearn-pipelines-grid-search-with-cross-validation-ccfc74b0ce9f

grid_result = search.fit(X_training, Y_training, callbacks= my_callbacks,validation_data=(X_test, Y_test) ) 

#,n_jobs=-1


# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
print ("rest")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 3 folds for each of 288 candidates, totalling 864 fits
[CV 1/3; 1/288] START LAMBDA=1e-07, decay=1e-06, dense_nparams=300, filters=10, lr=1e-06, optimizer=Adam
[CV 1/3; 1/288] END LAMBDA=1e-07, decay=1e-06, dense_nparams=300, filters=10, lr=1e-06, optimizer=Adam; total time=  49.1s
[CV 2/3; 1/288] START LAMBDA=1e-07, decay=1e-06, dense_nparams=300, filters=10, lr=1e-06, optimizer=Adam


In [ ]:
GridSearchCV(estimator=kears_estimator,  param_grid=param_grid,cv=kfold_splits)

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.xscale("log")
plt.yscale("log")
plt.xlim(50, 1000)
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
best = model #rnd_search_cv.best_estimator_
pred = best.predict(X_test)
pred = pred.flatten()
import matplotlib.pyplot as plt

#plt.plot(pred, Y_test, "+")
plt.hist(np.abs(pred - np.array(Y_test)))
plt.show()
np.mean(np.abs(pred - Y_test))  

In [ ]:
import tqdm

mc_predictions = []
for i in tqdm.tqdm(range(1000)):
    y_p = best(X_test, training=True)
    mc_predictions.append(y_p)
    
mc_predictions = np.array(mc_predictions)

In [ ]:
mc_predictions.shape

In [ ]:
mc_ensemble_pred = np.array(mc_predictions).mean(axis=0)

In [ ]:
mc_ensemble_pred


plt.hist(np.abs(mc_ensemble_pred.flatten() - np.array(Y_test)))
plt.show()
np.mean(np.abs(mc_ensemble_pred.flatten() - Y_test))  